##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFLite オーサリングツール

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/lite/guide/authoring">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     TensorFlow.org で表示</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/guide/authoring.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/guide/authoring.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     GitHubでソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/tensorflow/tensorflow/lite/g3doc/guide/authoring.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>

TensorFlow Lite Authoring API を使用すると、TensorFlow Lite と互換性がある `tf.function` モデルを管理できます。


## 設定

In [ ]:
import tensorflow as tf

## TensorFlow と TensorFlow Lite との互換性の問題

デバイスで TF モデルを使用する場合は、そのモデルを TFLite モデルに変換し、TFLite インタープリタから TFLite モデルを使用する必要があります。TFLite ビルトイン op セットによって TensorFlow op がサポートされていないと、変換中に互換性エラーが発生する可能性があります。

このような問題は厄介です。モデルのオーサリング時のように早い段階で検出するには、どうすれば良いでしょうか。

次のコードは、`converter.convert()` 呼び出し時に失敗します。


In [ ]:
@tf.function(input_signature=[
    tf.TensorSpec(shape=[None], dtype=tf.float32)
])
def f(x):
  return tf.cosh(x)

# Evaluate the tf.function
result = f(tf.constant([0.0]))
print (f"result = {result}")

In [ ]:
# Convert the tf.function
converter = tf.lite.TFLiteConverter.from_concrete_functions(
    [f.get_concrete_function()], f)
try:
  fb_model = converter.convert()
except Exception as e:
  print(f"Got an exception: {e}")

## シンプルなターゲット認識オーサリングの使用方法

モデルオーサリング時の TensorFlow Lite の互換性の問題を検出するために、Authoring API が導入されました。

必要な手順は、`@tf.lite.experimental.authoring.compatible` デコレータを追加し、`tf.function` モデルをラップして、TFLite の互換性を確認することだけです。

この後、モデルを評価するときに、互換性が自動的にチェックされます。

In [ ]:
@tf.lite.experimental.authoring.compatible
@tf.function(input_signature=[
    tf.TensorSpec(shape=[None], dtype=tf.float32)
])
def f(x):
  return tf.cosh(x)

# Evaluate the tf.function
result = f(tf.constant([0.0]))
print (f"result = {result}")


TensorFlow Lite の互換性の問題が検出された場合、`COMPATIBILITY WARNING` または `COMPATIBILITY ERROR` と、問題がある処理の正確な場所が表示されます。この例では、tf.function モデルの `tf.Cosh` 処理の場所が表示されています。

`<function_name>.get_compatibility_log()` メソッドを使用して、互換性ログを確認することもできます。

In [ ]:
compatibility_log = '\n'.join(f.get_compatibility_log())
print (f"compatibility_log = {compatibility_log}")

## 非互換性の場合に例外を発生させる

`@tf.lite.experimental.authoring.compatible` デコレータのオプションを指定できます。`raise_exception` オプションを使用すると、デコレートされたモデルを評価するときに、例外が発生します。

In [ ]:
@tf.lite.experimental.authoring.compatible(raise_exception=True)
@tf.function(input_signature=[
    tf.TensorSpec(shape=[None], dtype=tf.float32)
])
def f(x):
  return tf.cosh(x)

# Evaluate the tf.function
try:
  result = f(tf.constant([0.0]))
  print (f"result = {result}")
except Exception as e:
  print(f"Got an exception: {e}")

## "Select TF ops" の使用を指定する

[Select TF ops](https://www.tensorflow.org/lite/guide/ops_select) の使用をすでに認識している場合は、`converter_target_spec` を設定して、これを Authoring API に命令できます。これは、[tf.lite.TFLiteConverter](https://www.tensorflow.org/api_docs/python/tf/lite/TFLiteConverter) API で使用する [tf.lite.TargetSpec](https://www.tensorflow.org/api_docs/python/tf/lite/TargetSpec) オブジェクトと同じです。


In [ ]:
target_spec = tf.lite.TargetSpec()
target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS,
]
@tf.lite.experimental.authoring.compatible(converter_target_spec=target_spec, raise_exception=True)
@tf.function(input_signature=[
    tf.TensorSpec(shape=[None], dtype=tf.float32)
])
def f(x):
  return tf.cosh(x)

# Evaluate the tf.function
result = f(tf.constant([0.0]))
print (f"result = {result}")


## GPU 互換性の確認

モデルが TensorFlow Lite の [GPU デリゲート](https://www.tensorflow.org/lite/performance/gpu)と互換性があることを保証するには、[tf.lite.TargetSpec](https://www.tensorflow.org/api_docs/python/tf/lite/TargetSpec) の `experimental_supported_backends` を設定します。

次の例は、モデルの GPU デリゲートの互換性を保証する方法を示します。このモデルでは、tf.slice オペレータとサポートされていない tf.cosh オペレータが使用されているため、互換性の問題があります。2 件の`互換性の警告`とその場所に関する情報が表示されます。

In [ ]:
target_spec = tf.lite.TargetSpec()
target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS,
]
target_spec.experimental_supported_backends = ["GPU"]
@tf.lite.experimental.authoring.compatible(converter_target_spec=target_spec)
@tf.function(input_signature=[
    tf.TensorSpec(shape=[4, 4], dtype=tf.float32)
])
def func(x):
  y = tf.cosh(x)
  return y + tf.slice(x, [1, 1], [1, 1])

result = func(tf.ones(shape=(4,4), dtype=tf.float32))

## その他の資料

詳細については、次のドキュメントを参照してください。

- [tf.function](https://www.tensorflow.org/api_docs/python/tf/function) API ドキュメント
- [tf.function によるパフォーマンスの改善](https://www.tensorflow.org/guide/function)
- [TensorFlow Lite コンバータ](https://www.tensorflow.org/lite/models/convert)
- [TensorFlow Lite Model Analyzer](https://www.tensorflow.org/lite/guide/model_analyzer)
- [TensorFlow Lite GPU デリゲート](https://www.tensorflow.org/lite/performance/gpu)